# Phase 1 -  Data Wrangling 
###  First we need to collect Data.
# For  and explore each dataset, find any "bad formatation" or null value.


In [37]:
import pandas as pd
import os
import plotly.express as px
import matplotlib as plt
import warnings

warnings.filterwarnings("ignore")

# Dublin Bikes

In [38]:
path = "./"
files = [file for file in os.listdir(path) if file.startswith('dublinbike-historical-data-2023') ] # filtere only historical data files

all_hist = pd.DataFrame()

for file in files:
    current_month = pd.read_csv(path+"/"+file)
    all_hist = pd.concat([all_hist, current_month])



In [3]:
all_hist.head(10)

,STATION ID,TIME,LAST UPDATED,NAME,BIKE_STANDS,AVAILABLE_BIKE_STANDS,AVAILABLE_BIKES,STATUS,ADDRESS,LATITUDE,LONGITUDE
0,1,2023-01-01 00:00:03,2022-12-31 23:59:39,CLARENDON ROW,31,31,0,OPEN,Clarendon Row,53.3409,-6.26250
1,2,2023-01-01 00:00:03,2022-12-31 23:57:48,BLESSINGTON STREET,20,18,2,OPEN,Blessington Street,53.3568,-6.26814
2,3,2023-01-01 00:00:03,2022-12-31 23:57:10,BOLTON STREET,20,9,11,OPEN,Bolton Street,53.3512,-6.26986
3,4,2023-01-01 00:00:03,2022-12-31 23:51:39,GREEK STREET,20,8,12,OPEN,Greek Street,53.3469,-6.27298
4,5,2023-01-01 00:00:03,2022-12-31 23:58:28,CHARLEMONT PLACE,40,16,24,OPEN,Charlemont Street,53.3307,-6.26018
5,6,2023-01-01 00:00:03,2022-12-31 23:52:38,CHRISTCHURCH PLACE,20,1,19,OPEN,Christchurch Place,53.3434,-6.27012
6,7,2023-01-01 00:00:03,2022-12-31 23:54:28,HIGH STREET,29,27,2,OPEN,High Street,53.3436,-6.27507
7,8,2023-01-01 00:00:03,2022-12-31 23:58:33,CUSTOM HOUSE QUAY,30,27,3,OPEN,Custom House Quay,53.3479,-6.24805
8,9,2023-01-01 00:00:03,2022-12-31 23:56:04,EXCHEQUER STREET,24,20,4,OPEN,Exchequer Street,53.3430,-6.26358
9,10,2023-01-01 00:00:03,2022-12-31 23:55:56,DAME STREET,16,1,14,OPEN,Dame Street,53.3440,-6.26680


In [39]:
# From the dataset, the bike stations will be extracted buy removing duplicates stations from all hist
# also, unicessary columns will be removed/
bike_stations = all_hist[['STATION ID','NAME','BIKE_STANDS','STATUS','LATITUDE', 'LONGITUDE']].drop_duplicates(subset=["STATION ID"])
bike_stations.head()

,STATION ID,NAME,BIKE_STANDS,STATUS,LATITUDE,LONGITUDE
0,1,CLARENDON ROW,31,OPEN,53.3409,-6.26250
1,2,BLESSINGTON STREET,20,OPEN,53.3568,-6.26814
2,3,BOLTON STREET,20,OPEN,53.3512,-6.26986
3,4,GREEK STREET,20,OPEN,53.3469,-6.27298
4,5,CHARLEMONT PLACE,40,OPEN,53.3307,-6.26018


# For usage analysis, the closed station historic will be removeed, as well ass, the "Address", "last update, Longitude",Latitude.


In [40]:
# drop all rowns with STATUS - CLOSED

all_hist = all_hist[all_hist['STATUS'] == 'OPEN']
all_hist = all_hist[['STATION ID', 'TIME', 'NAME', 'BIKE_STANDS', 'AVAILABLE_BIKE_STANDS', 'AVAILABLE_BIKES']]

#rename column AVAILABLE_BIKE_STANDS to BIKES_IN_USE
all_hist.rename(columns={'AVAILABLE_BIKE_STANDS': 'BIKES_IN_USE'}, inplace=True)

In [6]:
all_hist

,STATION ID,TIME,NAME,BIKE_STANDS,BIKES_IN_USE,AVAILABLE_BIKES
0,1,2023-01-01 00:00:03,CLARENDON ROW,31,31,0
1,2,2023-01-01 00:00:03,BLESSINGTON STREET,20,18,2
2,3,2023-01-01 00:00:03,BOLTON STREET,20,9,11
3,4,2023-01-01 00:00:03,GREEK STREET,20,8,12
4,5,2023-01-01 00:00:03,CHARLEMONT PLACE,40,16,24
...,...,...,...,...,...,...
27469,113,2023-12-06 00:30:02,MERRION SQUARE SOUTH,40,21,17
27470,114,2023-12-06 00:30:02,WILTON TERRACE (PARK),40,40,0
27471,115,2023-12-06 00:30:02,KILLARNEY STREET,30,1,29
27472,116,2023-12-06 00:30:02,BROADSTONE,30,28,2


# Dublin Bikes Revies


In [ ]:
from dotenv import load_dotenv
from os import getenv

# Load from the .env
load_dotenv()

import requests
import json


# Get the environmental variables
API_KEY = getenv('API_KEY')

dublinbikes_id = "186605"

base_url = "https://api.tripadvisor.com/api/2/reviews/list"
params = {
    "locationId": dublinbikes_id,
    "currencyCode": "EUR",
    "ratings": "all"
}


headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "X-APIKEY": API_KEY
}


response = requests.get(base_url, params=params, headers=headers)


reviews = json.loads(response.content.decode())

for review in reviews["reviews"]:
    print(review["rating"], review["comment"])

In [ ]:
import requests
from bs4 import BeautifulSoup 


url_list = ['https://www.tripadvisor.com/Attraction_Review-g186605-d6439815-Reviews-Dublin_Bikes-Dublin_County_Dublin.html',
            'https://www.tripadvisor.com/Attraction_Review-g186605-d6439815-Reviews-or10-Dublin_Bikes-Dublin_County_Dublin.html',
            'https://www.tripadvisor.com/Attraction_Review-g186605-d6439815-Reviews-or20-Dublin_Bikes-Dublin_County_Dublin.html',
            'https://www.tripadvisor.com/Attraction_Review-g186605-d6439815-Reviews-or30-Dublin_Bikes-Dublin_County_Dublin.html',
            'https://www.tripadvisor.com/Attraction_Review-g186605-d6439815-Reviews-or40-Dublin_Bikes-Dublin_County_Dublin.html' ]

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}



In [ ]:
# Function getinfo retrieves reviews and their corresponding titles and ratings from  tripadvisor URL.
# it uses BeautifulSoup to scrape the website and stores the extracted data in a Pandas DataFrame.
# Args: url (str): The URL of the website containing the reviews to be extracted.
# Returns:  review_df (DataFrame): A Pandas DataFrame containing the title, reviews and ratings.

    
def getinfo(url):     
        
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
   
        review_df = pd.DataFrame()
        
        
        titles = soup.find_all('div', attrs={'class': 'biGQs _P fiohW qWPrE ncFvv fOtGX'})
        title_lst = []
        for title in titles:
             title_lst.append(title.find('span', attrs={'class': 'yCeTE'}).text)
        
        
        reviews = soup.find_all('span', attrs={'class':'JguWG'})

        review_list = []
        for review in reviews:
                review_list.append(review.find('span', attrs={'class': 'yCeTE'}).text)
                
        rate = soup.find_all('svg', attrs={'class':'UctUV d H0'})
        rate_list = []
        for r in rate:
                rate_list.append(r.get('aria-label'))
        
        review_df['title'] = pd.DataFrame(title_lst)
        review_df['review'] = pd.DataFrame(review_list)
        review_df['rate'] = pd.DataFrame(rate_list)
        
        
        return review_df
        
        
        


# this code will call the function "getinfo" for each url in the list "url_list"
# it will wait for 5 seconds between each call to the function "getinfo" ti aviud iverloading the server.

In [ ]:
import time
all_reviews_df = pd.DataFrame()
for url in url_list:
    time.sleep(5)
    try:
        all_reviews_df = pd.concat([all_reviews_df, getinfo(url)])    
    except Exception:
        print('Error to scrap url:', url)
        print('Scrapping again!')
        all_reviews_df = pd.concat([all_reviews_df, getinfo(url)])    
    

In [ ]:
all_reviews_df['rate'] = all_reviews_df['rate'].str.slice(0,3)


In [ ]:

all_reviews_df.reset_index(drop=True, inplace=True)
all_reviews_df.to_csv('all_reviews.csv', index=False)


In [ ]:
all_reviews_df = pd.read_csv("all_reviews.csv")

In [ ]:
display(all_reviews_df)

In [ ]:
grouped = all_reviews_df.groupby(by = 'rate').count().reset_index()

import plotly.express as px
# This dataframe has 244 lines, but 4 distinct values for `day`

fig = px.pie(grouped, values='review', 
             names='rate',
             title = "% Of reviews by rate"            )
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

# For  Bicing /stations in Barcelona, the dataset is in json format, 

In [ ]:
import pandas as pd
import json
import requests
url = "https://opendata-ajuntament.barcelona.cat/data/dataset/bd2462df-6e1e-4e37-8205-a4b8e7313b84/resource/e5adca8d-98bf-42c3-9b9c-364ef0a80494/download"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    stations = data['data']
    df = pd.DataFrame(stations['stations'])
    
else:
    print("Error:", response.status_code)
    
df.head(2)


In [ ]:

df.columns

In [ ]:
bicing_df = df[['station_id', 'name', 'address','lat', 'lon','capacity']]
bicing_df.head()

In [ ]:
# Convert nested JSON to DataFrame using json_normalize()
#df = pd.read_json(json_data)
#df = pd.json_normalize(json_data)
# Print the DataFrame
#print(df)

In [ ]:
import requests
import pandas as pd

url = "https://opendata-ajuntament.barcelona.cat/data/dataset/bd2462df-6e1e-4e37-8205-a4b8e7313b84/resource/e5adca8d-98bf-42c3-9b9c-364ef0a80494/download"




In [ ]:
all_hist.info()

In [ ]:
all_hist.info()

In [ ]:
all_hist.isnull().sum()


In [ ]:
#all_hist.reset_index()

In [36]:
all_hist['STATION ID'].nunique()

115

In [41]:
import plotly.express as px
color_mapper = {"OPEN": "#7CFC00", "CLOSED": "#DF0030"}

fig = px.scatter_mapbox(bike_stations, 
                        lat="LATITUDE",
                        lon="LONGITUDE", 
                        color="STATUS", 
                        size="BIKE_STANDS", 
                        color_discrete_map=color_mapper,
                        size_max=15, 
                        zoom=12, 
                        hover_data=["BIKE_STANDS"],
                        mapbox_style="open-street-map")

# Update layout for larger figure size
fig.update_layout(
    title="Bike Stations in Dublin",
    autosize=False,
    width=1200, 
    height=600,  
)
fig.show()

In [ ]:
fig = px.pie(bike_stations, values='BIKE_STANDS', names='STATUS')
fig.update_layout(
    title="Bike Stations in Dublin",
    autosize=False,
    width=400,  
    height=400, 
)

fig.show()

In [33]:
all_hist

# change the weekday from numeric to respective day of the week



In [67]:
import plotly.express as px
import pandas as pd

# Load the data
all_hist['TIME'] = pd.to_datetime(all_hist['TIME'])

# get the weekday from the time column
all_hist['weekday'] = all_hist['TIME'].dt.weekday

# filter all_hist dataframe by month usint the TIME column
all_hist['month'] = all_hist['TIME'].dt.month

all_hist_jan = all_hist[all_hist['TIME'].dt.month == 5]

# Group the data by hour and calculate the total number of bikes used
df_grouped = all_hist_jan.groupby('weekday')['BIKES_IN_USE'].sum()

# Create a bar plot
fig = px.line(df_grouped, x=df_grouped.index, y='BIKES_IN_USE', title='Bike Usage by Day of Week in Dublin')

# Rotate x-axis labels to avoid overlapping
fig.update_layout(xaxis_tickangle=45)

# Show the chart
fig.show()


In [19]:
#pip install dash

In [ ]:
bike_usage = all_hist[['TIME', 'STATION ID', 'NAME', 'BIKE_STANDS','AVAILABLE_BIKES']]

bike_usage['TIME'] = pd.to_datetime(bike_usage['TIME'])
bike_usage.set_index('TIME')

In [ ]:
bicing_df['capacity'].sum()

In [ ]:
bicing_df.head()

In [ ]:
df